# Setup

In [ ]:
import sys, os, json 
import string 

import pandas as pd 
import numpy as np 

# turn off the pd chained assignment warning 
pd.options.mode.chained_assignment = None 

folder_path = f"C:/Users/YOUR_FILE_PATH"

## Load Workbook 

In [2]:
import openpyxl 

# load the Excel workbook 
workbook_name = f"{folder_path}//Enter Lock-ins.xlsx" 
wb = openpyxl.load_workbook(workbook_name)  

# Import Data 

## Players 

In [3]:
# read in the players data 
df_players = pd.read_csv(f"{folder_path}//raw_players.csv") 

# put together a full name column 
df_players["FULL_NAME"] = (df_players["FIRST_NAME"] + " " + df_players["LAST_NAME"]).str.upper() 

# showcase the data 
df_players.head() 

,PLAYER_ID,TEAM_ID,FIRST_NAME,LAST_NAME,POSITION,ACTIVE,JERSEY,BIRTH_DATE,HEIGHT_FT,HEIGHT_IN,WEIGHT,FULL_NAME
0,385,1,Larry,Nance Jr.,F-C,True,22.0,1993-01-01,6.0,7.0,245,LARRY NANCE JR.
1,1046,1,Trae,Young,G,True,11.0,1998-09-19,6.0,1.0,164,TRAE YOUNG
2,1868,1,De'Andre,Hunter,F-G,True,12.0,1997-12-02,6.0,8.0,221,DE'ANDRE HUNTER
3,2327,1,Garrison,Mathews,G,True,25.0,NaN,NaN,NaN,NaN,GARRISON MATHEWS
4,2629,1,Onyeka,Okongwu,F-C,True,17.0,2000-12-11,6.0,8.0,240,ONYEKA OKONGWU


## Games 

In [4]:
# read in the games dimension table 
df_games = pd.read_csv(f"{folder_path}//all_games.csv") 

# make sure that the game date is formatted correctly 
df_games["GAME_DATE"] = pd.to_datetime(df_games["GAME_DATE"]) 

# showcase the data 
df_games.head() 

,GAME_ID,LEAGUE,GAME_DATE,HOME_ID,HOME_TEAM,GUEST_ID,GUEST_TEAM,WEEK_NUMBER
0,14115,standard,2024-10-22 17:30:00,2,Boston Celtics,24,New York Knicks,1
1,14116,standard,2024-10-22 20:00:00,17,Los Angeles Lakers,22,Minnesota Timberwolves,1
2,14117,standard,2024-10-23 17:00:00,10,Detroit Pistons,15,Indiana Pacers,1
3,14118,standard,2024-10-23 17:30:00,1,Atlanta Hawks,4,Brooklyn Nets,1
4,14119,standard,2024-10-23 17:30:00,20,Miami Heat,26,Orlando Magic,1


## Stats 

In [5]:
# read in the stats data 
df_stats = pd.read_csv(f"{folder_path}//all_stats.csv") 

# make sure that the game date is formatted correctly 
df_stats["GAME_DATE"] = pd.to_datetime(df_stats["GAME_DATE"]) 

# showcase the data 
df_stats.head() 

,GAME_ID,PLAYER_ID,TEAM_ID,POINTS,POS,MIN,FGM,FGA,FGP,FTM,...,DD,TD,FGMI,FTMI,TPMI,PB40,PB50,AB15,RB20,FANTASY_POINTS
0,14115,882,2,37,NaN,30,14,18,77.8,1,...,True,False,4,1,3,0,0,0,0,80.0
1,14115,248,2,11,NaN,26,4,7,57.1,0,...,False,False,3,0,2,0,0,0,0,29.5
2,14115,242,2,18,NaN,31,7,9,77.8,0,...,False,False,2,0,2,0,0,0,0,41.0
3,14115,897,2,24,NaN,27,8,13,61.5,2,...,False,False,5,0,4,0,0,0,0,47.0
4,14115,75,2,23,NaN,30,7,18,38.9,4,...,False,False,11,0,4,0,0,0,0,47.5


## Player Schedules 

In [6]:
# concatenate the home and away teams 
df_pgames = pd.concat([
    (
        df_games[["HOME_ID", "GAME_ID", "WEEK_NUMBER", "GAME_DATE"]]
        .rename(columns = {"HOME_ID": "TEAM_ID"})
    ), 
    (
        df_games[["GUEST_ID", "GAME_ID", "WEEK_NUMBER", "GAME_DATE"]]
        .rename(columns = {"GUEST_ID": "TEAM_ID"}) 
    ) 
]) 

# join in a few player attributes 
df_pgames = df_pgames.merge(
    df_players[["TEAM_ID", "PLAYER_ID", "FULL_NAME"]], 
    on = "TEAM_ID", 
    how = "inner"
) 

# join in the stats if they exist 
df_pgames = df_pgames.merge(
    df_stats[["GAME_ID", "PLAYER_ID", "FANTASY_POINTS"]], 
    on = ["GAME_ID", "PLAYER_ID"], 
    how = "left" 
) 

# showcase the data 
df_pgames.head() 

,TEAM_ID,GAME_ID,WEEK_NUMBER,GAME_DATE,PLAYER_ID,FULL_NAME,FANTASY_POINTS
0,2,14115,1,2024-10-22 17:30:00,75,JAYLEN BROWN,47.5
1,2,14115,1,2024-10-22 17:30:00,882,JAYSON TATUM,80.0
2,2,14115,1,2024-10-22 17:30:00,897,DERRICK WHITE,47.0
3,2,14115,1,2024-10-22 17:30:00,819,LUKE KORNET,15.5
4,2,14115,1,2024-10-22 17:30:00,1038,LONNIE WALKER IV,NaN


# Define Functions 

## get_player_id 

In [7]:
# function to retrieve the player id based on the full name 
def get_player_id(sheet, name_cell):

    # get the player name 
    player_name = sheet[name_cell].value 

    # filter to the current player by name 
    current_player = df_players.loc[df_players["FULL_NAME"] == player_name.upper()] 

    # get the player id if the player could be found 
    if len(current_player.index) > 0:
        player_id = int(current_player["PLAYER_ID"].values[0]) 
    
    # otherwise, return None with an error message 
    else:
        print(f"Player {player_name} not found!") 
        sheet[name_cell].value = f"ERROR: Player '{player_name}' Not Found!" 
        player_id = None 
    
    return player_name, player_id 

# test the function 
get_player_id(
    sheet = wb["Week 14"], 
    name_cell = "B12" 
) 

('Fred VanVleet', 527)

## update_player_stats 

In [8]:
# function to update the stats for a given player 
def update_player_stats(sheet, player_id, week_number, base_row, cols):

    # filter the stats data for this player/week 
    current_stats = df_pgames.loc[
        (df_pgames["PLAYER_ID"] == player_id) &
        (df_pgames["WEEK_NUMBER"] == week_number) 
    ] 

    # add the day number column 
    current_stats["DAY_NUM"] = current_stats["GAME_DATE"].dt.dayofweek 

    # loop through each day of the week 
    for i in range(7): 
        day_stats = current_stats.loc[current_stats["DAY_NUM"] == i] 

        # date and time cells 
        date_cell = f"{cols[i + 1]}{base_row - 2}" 
        time_cell = f"{cols[i + 1]}{base_row - 1}" 
        stat_cell = f"{cols[i + 1]}{base_row}" 

        # fill in the info if there was a game this day 
        if len(day_stats.index) > 0:

            # get the date/stats 
            game_date = pd.to_datetime(day_stats["GAME_DATE"].values[0]) 
            game_stat = day_stats["FANTASY_POINTS"].values[0]  

            # fill in blanks for null stats 
            if np.isnan(game_stat):
                game_stat = "" 

            # fill in the info 
            sheet[date_cell].value = game_date.strftime("%a %m/%d") 
            sheet[time_cell].value = game_date.strftime("%I:%M %p") 
            sheet[stat_cell].value = game_stat 
        
        # otherwise, fill in blanks 
        else:
            sheet[date_cell].value = "" 
            sheet[time_cell].value = "" 
            sheet[stat_cell].value = "" 

# test the function 
update_player_stats(
    sheet = wb["Week 14"], 
    player_id = 527, 
    week_number = 14, 
    base_row = 12, 
    cols = string.ascii_uppercase[1:9] 
) 
print(wb["Week 14"]["C10"].value)
print(wb["Week 14"]["C11"].value)
print(wb["Week 14"]["C12"].value)

Mon 01/20
12:00 PM
43.0


## get_locked_points 

In [ ]:
# funtion to get the locked points based on the spreadsheet 
def get_locked_points(sheet, base_row, cols):

    # placeholder for the locked value 
    locked_points = np.nan  

    # loop through each day of the week and check for locked games 
    for i, c in enumerate(cols[2:]): 
        lock_val = sheet[f"{c}{base_row+1}"].value 
        if (lock_val is not None) and (lock_val != ""): 
            locked_points = float(sheet[f"{c}{base_row}"].value)  
            break 
    
    return locked_points 

get_locked_points(
    sheet = wb["Week 14"], 
    base_row = 7, 
    cols = string.ascii_uppercase[1:9] 
) 

82.5

# Applying the Functions 

In [10]:
# columns for our team vs the opponent 
team_cols = {
    "My Team": string.ascii_uppercase[1:9], 
    "Opponent": string.ascii_uppercase[10:18] 
} 

# other constants 
nplayers = 9  
row_start = 7 
rows_between = 5 

# placeholder for the locked points 
df = pd.DataFrame() 

# loop through each sheet 
for i, sheetname in enumerate(wb.sheetnames):
    if "Week" in sheetname:

        # get the current sheet and week number 
        sheet = wb[sheetname] 
        week_number = int(sheetname.split()[-1]) 

        # loop through the different teams 
        for tname, cols in team_cols.items():

            # loop through each of the players 
            for pnum in range(nplayers):
                base_row = row_start + (pnum * rows_between)  

                # get the player name and id 
                player_name, player_id = get_player_id(
                    sheet = sheet, 
                    name_cell = f"{cols[0]}{base_row}" 
                ) 

                # make sure that the player was found 
                if "ERROR" not in player_name:

                    # update the player stats 
                    update_player_stats(sheet, player_id, week_number, base_row, cols) 

                    # get the locked points 
                    locked_points = get_locked_points(sheet, base_row, cols) 

                    # update the locked points dataframe 
                    df = pd.concat([df, pd.DataFrame({
                        "WEEK_NUMBER": [week_number], 
                        "TEAM": tname, 
                        "PLAYER_ID": player_id, 
                        "PLAYER_NAME": player_name, 
                        "LOCKED_POINTS": locked_points 
                    })], ignore_index = True) 

# save the updated workbook 
wb.save(f"{folder_path}//Enter Lock-ins.xlsx") 

# save the locked points dataframe 
df.to_csv(f"{folder_path}//locked_points.csv", index = False) 

# showcase the data 
df.head() 

Player ERROR: Player 'ERROR: Player 'ERROR: Player 'ERROR: Player 'ERROR: Player 'ERROR: Player 'ERROR: Player 'ERROR: Player 'ERROR: Player 'ERROR: Player 'ERROR: Player 'ERROR: Player 'ERROR: Player 'ERROR: Player 'ERROR: Player 'ERROR: Player 'Damien Lillard' Not Found!' Not Found!' Not Found!' Not Found!' Not Found!' Not Found!' Not Found!' Not Found!' Not Found!' Not Found!' Not Found!' Not Found!' Not Found!' Not Found!' Not Found!' Not Found! not found!
Player ERROR: Player 'ERROR: Player 'ERROR: Player 'ERROR: Player 'ERROR: Player 'ERROR: Player 'ERROR: Player 'ERROR: Player 'ERROR: Player 'ERROR: Player 'ERROR: Player 'ERROR: Player 'ERROR: Player 'ERROR: Player 'ERROR: Player 'ERROR: Player 'Jaren Jackson' Not Found!' Not Found!' Not Found!' Not Found!' Not Found!' Not Found!' Not Found!' Not Found!' Not Found!' Not Found!' Not Found!' Not Found!' Not Found!' Not Found!' Not Found!' Not Found! not found!


,WEEK_NUMBER,TEAM,PLAYER_ID,PLAYER_NAME,LOCKED_POINTS
0,13,My Team,2595,Tyrese Haliburton,36.5
1,13,My Team,3406,Dyson Daniels,23.0
2,13,My Team,2801,Cade Cunningham,29.0
3,13,My Team,940,Mikal Bridges,32.5
4,13,My Team,882,Jayson Tatum,47.5
